<a href="https://colab.research.google.com/github/CarolineLaure/One_Qubit_TensorFlow_example/blob/master/two_level_system_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Optimization of two-level system. Let's have the Hamiltonian of the form H=-($\Omega$/2)$\sigma x$. The goald is to find a set of control parameters $\Omega$ which will maximize our fidelity


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import time
print(tf.__version__) 

2.2.0-rc2


In [0]:
Omega_1 = 2 * np.pi * 60 * 10**6

start = time.time()

In [13]:
class Propagator:
    def __init__(self, no_of_steps, dim, delta_t):
        self.delta_t=delta_t
        self.dim=2
        self.initial_state= tf.constant([1, 0], dtype=tf.complex128)
        self.final_state= tf.constant([0, 1], dtype=tf.complex128)
        self.sigma_x=tf.constant([[0, 1], [1, 0]], dtype=tf.complex128) # generation of $\sigma x$

        self.ctrl_amplitude = tf.Variable(tf.zeros([no_of_steps, 1], dtype=tf.float64), dtype=tf.float64)  # control amplitude $\Omega$

        self.generators =  tf.stack([
                                     tf.constant(Omega_1 * self.sigma_x, dtype=tf.complex128),
                                     
                                     ])

        self.contraction_array=[]
        contraction_array_length = int(np.floor(np.log2(no_of_steps)))
        temp_no_of_steps= no_of_steps

        for i in range(contraction_array_length):
          self.contraction_array.append(bool(np.mod(temp_no_of_steps, 2))) 
          temp_no_of_steps = np.floor(temp_no_of_steps/2)

    def exponential(self):
          regularize_amplitudes = 1/np.sqrt(2)*tf.math.tanh(self.ctrl_amplitude)
          exponent = -1j*self.delta_t*(0.5)*(tf.linalg.tensordot(tf.cast(regularize_amplitudes, dtype=tf.complex128), self.generators, 1))
          return tf.linalg.expm(exponent)

    def propagate(self):
          step_exps=self.exponential()
          for is_odd in self.contraction_array:
            if is_odd:
              odd_exp=step_exps[-1, :, :]
              step_exps = tf.linalg.matmul(step_exps[1::2, :, :], step_exps[0:-1:2, :, :])
              step_exps = tf.concat([step_exps[0:-1, :, :], [tf.linalg.matmul(odd_exp, step_exps[-1, :, :])]], 0)
            else:
              step_exps = tf.linalg.matmul(step_exps[1::2, :, :], step_exps[0::2, :, :])
          return tf.squeeze(step_exps)

    @tf.function
    def infidelity(self):
          propagator=self.propagate()
          intermediate_state = tf.linalg.matvec(propagator, tf.transpose(self.initial_state))
          overlap = tf.tensordot(tf.math.conj(intermediate_state), self.final_state, axes=1)
          return 1-tf.math.real(tf.math.conj(overlap)*overlap)

propagator = Propagator(5000, 2, 10**-9)

optimizer = tf.keras.optimizers.Adam(0.01)

propagator.ctrl_amplitude.assign(
    tf.random.uniform([5000, 1], -1, 1, dtype=tf.float64)
)

#propagator.infidelity()

@tf.function  
def optimization_step():
  with tf.GradientTape() as tape:
    infidelity = propagator.infidelity()
  gradients = tape.gradient(infidelity, [propagator.ctrl_amplitude])
  optimizer.apply_gradients(zip(gradients, [propagator.ctrl_amplitude]))
  return infidelity


steps = range(100)
for step in steps:
  current_infidelity = optimization_step()
  print('step %2d: infidelity=%2.5f' % (step, current_infidelity))



#final_pulse = propagator.ctrl_amplitude  



end = time.time()
print('Run Time: %2.4f seconds' %(end-start))

step  0: infidelity=0.61627
step  1: infidelity=0.73323
step  2: infidelity=0.70946
step  3: infidelity=0.57194
step  4: infidelity=0.31769
step  5: infidelity=0.54113
step  6: infidelity=0.51266
step  7: infidelity=0.05729
step  8: infidelity=0.40291
step  9: infidelity=0.02927
step 10: infidelity=0.30441
step 11: infidelity=0.19320
step 12: infidelity=0.08519
step 13: infidelity=0.24439
step 14: infidelity=0.00222
step 15: infidelity=0.18986
step 16: infidelity=0.08468
step 17: infidelity=0.06748
step 18: infidelity=0.14265
step 19: infidelity=0.00044
step 20: infidelity=0.12460
step 21: infidelity=0.02957
step 22: infidelity=0.06181
step 23: infidelity=0.07226
step 24: infidelity=0.00962
step 25: infidelity=0.08005
step 26: infidelity=0.00194
step 27: infidelity=0.05872
step 28: infidelity=0.02019
step 29: infidelity=0.02847
step 30: infidelity=0.03616
step 31: infidelity=0.00711
step 32: infidelity=0.03982
step 33: infidelity=0.00007
step 34: infidelity=0.03398
step 35: infidelity=